# K₇ Spectral Convergence: Is 14 Real or Sweet Spot?

## Question

Previous run showed k=50, N=10000 → λ₁×H* = 14.03

Is this:
1. **Real convergence** to 14 = dim(G₂) (Pell exact)?
2. **Sweet spot** that happens to hit 14 by accident?

## Test Strategy

1. **Fine k-grid around 50**: k ∈ {30, 40, 50, 60, 70}
2. **Multiple N values**: N ∈ {10k, 15k, 20k}
3. **Multiple seeds**: 5 seeds per (N, k) for stability
4. **Check monotonicity**: Does λ₁×H* decrease monotonically with N?

**Verdict criteria**:
- If ALL (N, k) combinations converge toward 14 → **Real**
- If only k=50 hits 14 → **Sweet spot**

---

In [ ]:
import numpy as np
from scipy.sparse import csr_matrix, diags
from scipy.sparse.linalg import eigsh
import json
from datetime import datetime
import time
import gc

# GIFT Constants
H_STAR = 99
DIM_G2 = 14
DET_G = 65/32
RATIO = H_STAR / 84

print("K₇ Convergence Validation")
print("=" * 50)
print(f"Target: λ₁×H* = {DIM_G2} (dim G₂)")
print(f"H* = {H_STAR}")

In [ ]:
def sample_K7(N, seed):
    """Sample K₇ via TCS."""
    rng = np.random.default_rng(seed)
    theta = rng.uniform(0, 2*np.pi, N).astype(np.float32)
    q1 = rng.standard_normal((N, 4)).astype(np.float32)
    q1 /= np.linalg.norm(q1, axis=1, keepdims=True)
    q2 = rng.standard_normal((N, 4)).astype(np.float32)
    q2 /= np.linalg.norm(q2, axis=1, keepdims=True)
    return theta, q1, q2


def compute_lambda1(theta, q1, q2, k, batch_size=2500):
    """Compute λ₁ for K₇ with given k."""
    N = len(theta)
    alpha = DET_G / (RATIO ** 3)
    
    # Build kNN graph in batches
    all_neighbors = []
    all_distances = []
    
    for i_start in range(0, N, batch_size):
        i_end = min(i_start + batch_size, N)
        batch_n = i_end - i_start
        
        D_batch = np.zeros((batch_n, N), dtype=np.float32)
        
        for i_loc, i_glob in enumerate(range(i_start, i_end)):
            diff_theta = np.abs(theta[i_glob] - theta)
            d_S1 = np.minimum(diff_theta, 2*np.pi - diff_theta)
            
            dot1 = np.clip(np.abs(np.sum(q1[i_glob] * q1, axis=1)), -1, 1)
            d_S3_1 = 2 * np.arccos(dot1)
            
            dot2 = np.clip(np.abs(np.sum(q2[i_glob] * q2, axis=1)), -1, 1)
            d_S3_2 = 2 * np.arccos(dot2)
            
            D_batch[i_loc] = np.sqrt(alpha * d_S1**2 + d_S3_1**2 + RATIO**2 * d_S3_2**2)
        
        k_actual = min(k + 1, N)
        idx = np.argpartition(D_batch, k_actual, axis=1)[:, :k_actual]
        dists = np.take_along_axis(D_batch, idx, axis=1)
        
        sort_idx = np.argsort(dists, axis=1)
        idx = np.take_along_axis(idx, sort_idx, axis=1)[:, 1:k+1]
        dists = np.take_along_axis(dists, sort_idx, axis=1)[:, 1:k+1]
        
        all_neighbors.append(idx)
        all_distances.append(dists)
        
        del D_batch
    
    neighbors = np.vstack(all_neighbors)
    distances = np.vstack(all_distances)
    
    sigma = float(np.median(distances[:, -1]))
    
    row = np.repeat(np.arange(N), k)
    col = neighbors.flatten()
    weights = np.exp(-distances.flatten()**2 / (2 * sigma**2))
    
    W = csr_matrix((weights, (row, col)), shape=(N, N))
    W = (W + W.T) / 2
    
    d = np.array(W.sum(axis=1)).flatten()
    d_inv_sqrt = 1.0 / np.sqrt(d + 1e-10)
    D_inv_sqrt = diags(d_inv_sqrt)
    L = diags(np.ones(N)) - D_inv_sqrt @ W @ D_inv_sqrt
    
    eigvals, _ = eigsh(L.tocsr(), k=3, which='SM', tol=1e-8)
    lambda1 = float(sorted(eigvals)[1])
    
    del W, L, neighbors, distances
    gc.collect()
    
    return lambda1, sigma

print("✓ Functions ready")

In [ ]:
# Configuration
N_VALUES = [10000, 15000, 20000]
K_VALUES = [30, 40, 50, 60, 70]
N_SEEDS = 3

print("Configuration:")
print(f"  N: {N_VALUES}")
print(f"  k: {K_VALUES}")
print(f"  Seeds: {N_SEEDS}")
print(f"  Total runs: {len(N_VALUES) * len(K_VALUES) * N_SEEDS}")

In [ ]:
# Main computation
print("\n" + "=" * 70)
print("CONVERGENCE STUDY")
print("=" * 70)

results = []

for N in N_VALUES:
    print(f"\nN = {N}")
    print("-" * 60)
    print(f"{'k':>5} | {'λ₁×H* (mean)':>14} | {'std':>8} | {'min':>8} | {'max':>8}")
    print("-" * 60)
    
    for k in K_VALUES:
        seed_results = []
        
        for seed in range(N_SEEDS):
            theta, q1, q2 = sample_K7(N, seed=42 + seed)
            lambda1, sigma = compute_lambda1(theta, q1, q2, k)
            product = lambda1 * H_STAR
            seed_results.append(product)
            
            results.append({
                'N': int(N),
                'k': int(k),
                'seed': int(seed),
                'lambda1': float(lambda1),
                'lambda1_H': float(product),
                'sigma': float(sigma)
            })
            
            del theta, q1, q2
            gc.collect()
        
        mean_val = np.mean(seed_results)
        std_val = np.std(seed_results)
        min_val = np.min(seed_results)
        max_val = np.max(seed_results)
        
        # Highlight if close to 14
        marker = "← 14!" if abs(mean_val - 14) < 1 else ""
        
        print(f"{k:>5} | {mean_val:>14.2f} | {std_val:>8.3f} | {min_val:>8.2f} | {max_val:>8.2f} {marker}")

In [ ]:
# Analysis: Is it a sweet spot?
print("\n" + "=" * 70)
print("SWEET SPOT ANALYSIS")
print("=" * 70)

# For each k, check if λ₁×H* decreases monotonically with N
print("\nMonotonicity check (λ₁×H* should decrease with N):")
print("-" * 50)

monotonic_count = 0
converges_to_14 = 0

for k in K_VALUES:
    means_by_N = []
    for N in N_VALUES:
        vals = [r['lambda1_H'] for r in results if r['N'] == N and r['k'] == k]
        means_by_N.append(np.mean(vals))
    
    # Check monotonicity
    is_monotonic = all(means_by_N[i] >= means_by_N[i+1] for i in range(len(means_by_N)-1))
    monotonic_count += int(is_monotonic)
    
    # Check if converging toward 14
    final_val = means_by_N[-1]
    near_14 = abs(final_val - 14) < 2
    converges_to_14 += int(near_14)
    
    status = "✓" if is_monotonic else "✗"
    target = "→14" if near_14 else f"→{final_val:.0f}"
    
    print(f"  k={k:>3}: {' → '.join(f'{v:.1f}' for v in means_by_N)} [{status} mono] [{target}]")

print(f"\nMonotonic: {monotonic_count}/{len(K_VALUES)}")
print(f"Converges near 14: {converges_to_14}/{len(K_VALUES)}")

In [ ]:
# Extrapolation to N→∞
print("\n" + "=" * 70)
print("EXTRAPOLATION TO N→∞")
print("=" * 70)

print("\nFitting λ₁×H* = a + b/N for each k:")
print("-" * 50)

extrapolations = {}

for k in K_VALUES:
    Ns = []
    vals = []
    for N in N_VALUES:
        v = [r['lambda1_H'] for r in results if r['N'] == N and r['k'] == k]
        Ns.append(N)
        vals.append(np.mean(v))
    
    # Fit: λ₁×H* = a + b/N
    # Using 1/N as x-variable
    x = 1.0 / np.array(Ns)
    y = np.array(vals)
    
    # Linear regression
    A = np.vstack([np.ones_like(x), x]).T
    coeffs, residuals, rank, s = np.linalg.lstsq(A, y, rcond=None)
    a, b = coeffs
    
    # a is the N→∞ limit
    extrapolations[k] = {'limit': float(a), 'slope': float(b)}
    
    # Predicted values for comparison
    y_pred = a + b * x
    r2 = 1 - np.sum((y - y_pred)**2) / np.sum((y - np.mean(y))**2) if len(y) > 1 else 0
    
    marker = "← 14!" if abs(a - 14) < 1 else ""
    print(f"  k={k:>3}: limit = {a:>6.2f}, slope = {b:>8.0f}, R² = {r2:.3f} {marker}")

# Summary
limits = [extrapolations[k]['limit'] for k in K_VALUES]
print(f"\nMean extrapolated limit: {np.mean(limits):.2f} ± {np.std(limits):.2f}")

In [ ]:
# Visualization
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Plot 1: λ₁×H* vs N for each k
ax = axes[0]
colors = plt.cm.viridis(np.linspace(0.2, 0.8, len(K_VALUES)))

for i, k in enumerate(K_VALUES):
    means = []
    stds = []
    for N in N_VALUES:
        vals = [r['lambda1_H'] for r in results if r['N'] == N and r['k'] == k]
        means.append(np.mean(vals))
        stds.append(np.std(vals))
    
    ax.errorbar(N_VALUES, means, yerr=stds, marker='o', color=colors[i], 
                label=f'k={k}', capsize=3, linewidth=2)

ax.axhline(14, color='red', linestyle='--', linewidth=2, label='Target (14)')
ax.axhline(13, color='blue', linestyle=':', alpha=0.5, label='Alt (13)')
ax.set_xlabel('N', fontsize=12)
ax.set_ylabel('λ₁ × H*', fontsize=12)
ax.set_title('Convergence with N', fontsize=14)
ax.legend(loc='upper right')
ax.grid(True, alpha=0.3)

# Plot 2: Extrapolated limits
ax = axes[1]
limits = [extrapolations[k]['limit'] for k in K_VALUES]
ax.bar(K_VALUES, limits, color='steelblue', alpha=0.7, edgecolor='black')
ax.axhline(14, color='red', linestyle='--', linewidth=2, label='Target (14)')
ax.axhline(13, color='blue', linestyle=':', alpha=0.5, label='Alt (13)')

for k, lim in zip(K_VALUES, limits):
    ax.text(k, lim + 0.3, f'{lim:.1f}', ha='center', fontsize=10, fontweight='bold')

ax.set_xlabel('k', fontsize=12)
ax.set_ylabel('Extrapolated λ₁×H* (N→∞)', fontsize=12)
ax.set_title('Extrapolation to N→∞', fontsize=14)
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

# Plot 3: Stability (std across seeds)
ax = axes[2]
for i, k in enumerate(K_VALUES):
    stds = []
    for N in N_VALUES:
        vals = [r['lambda1_H'] for r in results if r['N'] == N and r['k'] == k]
        stds.append(np.std(vals))
    ax.plot(N_VALUES, stds, 'o-', color=colors[i], label=f'k={k}', linewidth=2)

ax.set_xlabel('N', fontsize=12)
ax.set_ylabel('Std(λ₁×H*) across seeds', fontsize=12)
ax.set_title('Stability (lower = better)', fontsize=14)
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('K7_Convergence_Validation.png', dpi=150, bbox_inches='tight')
plt.show()
print("✓ Saved: K7_Convergence_Validation.png")

In [ ]:
# Final verdict
print("\n" + "=" * 70)
print("FINAL VERDICT")
print("=" * 70)

mean_limit = np.mean(limits)
std_limit = np.std(limits)

print(f"\nExtrapolated N→∞ limits across all k:")
print(f"  Mean: {mean_limit:.2f}")
print(f"  Std:  {std_limit:.2f}")
print(f"  Range: [{min(limits):.2f}, {max(limits):.2f}]")

# Decision
if std_limit < 1 and abs(mean_limit - 14) < 1:
    print("\n" + "="*50)
    print("  VERDICT: ✓ REAL CONVERGENCE TO 14")
    print("="*50)
    print("  All k values extrapolate to ~14")
    print("  This is NOT a sweet spot")
    print(f"  λ₁ × H* = dim(G₂) = 14 is CONFIRMED")
    verdict = "CONFIRMED_14"
elif std_limit < 1 and abs(mean_limit - 13) < 1:
    print("\n" + "="*50)
    print("  VERDICT: ✓ REAL CONVERGENCE TO 13")
    print("="*50)
    print(f"  λ₁ × H* = dim(G₂) - 1 = 13")
    verdict = "CONFIRMED_13"
elif std_limit > 2:
    print("\n" + "="*50)
    print("  VERDICT: ⚠️ SWEET SPOT / UNSTABLE")
    print("="*50)
    print("  Different k give different limits")
    print("  Need better method")
    verdict = "SWEET_SPOT"
else:
    print("\n" + "="*50)
    print("  VERDICT: ❓ INCONCLUSIVE")
    print("="*50)
    print(f"  Mean limit: {mean_limit:.2f}")
    print("  Need larger N or more data")
    verdict = "INCONCLUSIVE"

In [ ]:
# Save results
output = {
    'metadata': {
        'notebook': 'K7_Convergence_Validation.ipynb',
        'timestamp': datetime.now().isoformat(),
        'N_values': N_VALUES,
        'k_values': K_VALUES,
        'n_seeds': N_SEEDS
    },
    'results': results,
    'extrapolations': extrapolations,
    'summary': {
        'mean_limit': float(mean_limit),
        'std_limit': float(std_limit),
        'verdict': verdict
    }
}

with open('K7_Convergence_Validation_results.json', 'w') as f:
    json.dump(output, f, indent=2)

print(f"\n✓ Saved: K7_Convergence_Validation_results.json")

---

## Interpretation Guide

| Verdict | Meaning | Next Step |
|---------|---------|------------|
| CONFIRMED_14 | All k converge to 14 | Celebrate! Pell exact |
| CONFIRMED_13 | All k converge to 13 | Spinor correction real |
| SWEET_SPOT | Only some k hit target | Method is unreliable |
| INCONCLUSIVE | Need more data | Run with larger N |

---

*GIFT K₇ Spectral Validation*